In [2]:
import pathlib

import numpy as np
from rich.pretty import pprint

from oqd_dataschema.base import Dataset
from oqd_dataschema.datastore import Datastore
from oqd_dataschema.groups import (
    ExpectationValueDataGroup,
    MeasurementOutcomesDataGroup,
    SinaraRawDataGroup,
)

In [3]:
raw = SinaraRawDataGroup(
    camera_images=Dataset(shape=(3, 2, 2), dtype="float32"),
    attrs={"date": "2025-03-26", "version": 0.1},
)
pprint(raw)

SinaraRawDataGroup(
│   attrs={'date': '2025-03-26', 'version': 0.1},
│   camera_images=Dataset(dtype='float32', shape=(3, 2, 2), data=None, attrs={})
)

In [4]:
raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float32")
pprint(raw)

SinaraRawDataGroup(
│   attrs={'date': '2025-03-26', 'version': 0.1},
│   camera_images=Dataset(
│   │   dtype='float32',
│   │   shape=(3, 2, 2),
│   │   data=array([[[0.34812397, 0.3357875 ],
│   │   [0.8069962 , 0.1842921 ]],
│      
│      [[0.1306957 , 0.70463485],
│   │   [0.10835699, 0.4392347 ]],
│      
│      [[0.61608946, 0.05809297],
│   │   [0.5589387 , 0.43851128]]], dtype=float32),
│   │   attrs={}
│   )
)

In [5]:
# raw.camera_images.data = np.random.uniform(size=(3, 4, 4)).astype("float32")
# raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float64")
raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float32")

In [6]:
data = Datastore(groups={"raw": raw})
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.5657244 , 0.74855775],
│   │   [0.01921861, 0.59149003]],
│      
│      [[0.18522406, 0.34376383],
│   │   [0.20763582, 0.71294636]],
│      
│      [[0.93185776, 0.6862982 ],
│   │   [0.5838989 , 0.180811  ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   )
│   }
)

In [7]:
def process_raw(raw: SinaraRawDataGroup) -> MeasurementOutcomesDataGroup:
    processed = MeasurementOutcomesDataGroup(
        outcomes=Dataset(
            data=np.round(raw.camera_images.data.mean(axis=(1, 2))),
        )
    )
    return processed


processed = process_raw(data.groups["raw"])
pprint(processed)

MeasurementOutcomesDataGroup(
│   attrs={},
│   outcomes=Dataset(dtype='float32', shape=(3,), data=array([0., 0., 1.], dtype=float32), attrs={})
)

In [8]:
data.groups.update(processed=processed)
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.5657244 , 0.74855775],
│   │   [0.01921861, 0.59149003]],
│      
│      [[0.18522406, 0.34376383],
│   │   [0.20763582, 0.71294636]],
│      
│      [[0.93185776, 0.6862982 ],
│   │   [0.5838989 , 0.180811  ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(
│   │   │   attrs={},
│   │   │   outcomes=Dataset(dtype='float32', shape=(3,), data=array([0., 0., 1.], dtype=float32), attrs={})
│   │   )
│   }
)

In [9]:
def process_outcomes(
    measurements: MeasurementOutcomesDataGroup,
) -> ExpectationValueDataGroup:
    expval = ExpectationValueDataGroup(
        expectation_value=Dataset(
            shape=(),
            dtype="float32",
            data=measurements.outcomes.data.mean(),
            attrs={"date": "20", "input": 10},
        )
    )
    return expval


expval = process_outcomes(processed)
data.groups.update(expval=process_outcomes(data.groups["processed"]))

pprint(expval)

ExpectationValueDataGroup(
│   attrs={},
│   expectation_value=Dataset(
│   │   dtype='float32',
│   │   shape=(),
│   │   data=np.float32(0.33333334),
│   │   attrs={'date': '20', 'input': 10}
│   )
)

In [10]:
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.5657244 , 0.74855775],
│   │   [0.01921861, 0.59149003]],
│      
│      [[0.18522406, 0.34376383],
│   │   [0.20763582, 0.71294636]],
│      
│      [[0.93185776, 0.6862982 ],
│   │   [0.5838989 , 0.180811  ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(
│   │   │   attrs={},
│   │   │   outcomes=Dataset(dtype='float32', shape=(3,), data=array([0., 0., 1.], dtype=float32), attrs={})
│   │   ),
│   │   'expval': ExpectationValueDataGroup(
│   │   │   attrs={},
│   │   │   expectation_value=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(),
│   │   │   │   data=np.float32(0.33333334),
│   │   │   │   attrs={'date': '20', 'input': 10}
│   │   │   )
│   │   )
│   }
)

In [11]:
filepath = pathlib.Path("test.h5")
data.model_dump_hdf5(filepath)

In [12]:
data_reload = Datastore.model_validate_hdf5(filepath)
pprint(data_reload)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=array([[[0.5657244 , 0.74855775],
│   │   [0.01921861, 0.59149003]],
│      
│      [[0.18522406, 0.34376383],
│   │   [0.20763582, 0.71294636]],
│      
│      [[0.93185776, 0.6862982 ],
│   │   [0.5838989 , 0.180811  ]]], dtype=float32)
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(attrs={}, outcomes=array([0., 0., 1.], dtype=float32)),
│   │   'expval': ExpectationValueDataGroup(attrs={}, expectation_value=array(0.33333334, dtype=float32))
│   }
)